### TODO:    
[x] SoftmaxLinearRegression    
[ ] Neural Nets    
[x] Class for scoring generating report stuff   
[x] hyperparameter tuning function    
[ ] Majority voting class (hard+soft)    
[ ] Stacking class    

In [132]:
import dill

In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

%matplotlib inline

/home/wolterlw/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [90]:
### loading data
with h5py.File('../data/mnist.hdf5','r') as f:
    Xtr = f['Xtr'].value.reshape(-1, 28*28)
    ytr = f['ytr'].value
    
    Xv = f['Xv'].value.reshape(-1, 28*28)
    yv = f['yv'].value
    
    Xts = f['Xts'].value.reshape(-1, 28*28)
    yts = f['yts'].value

with h5py.File('../data/usp_processed.hdf5', 'r') as f:
    Xusps = f['X'].value.reshape(-1, 28*28)
    yusps = f['y'].value

In [94]:
class Evaluator():
    def __init__(self, Xtr, ytr, Xv, yv, metrics={}):
        self.dat_tr = (Xtr, ytr)
        self.dat_v = (Xv, yv)
        self.metrics = metrics
        
    def score(self, model):
        pred_tr = model.predict(self.dat_tr[0])
        pred_v = model.predict(self.dat_v[0])
        
        res = {}
        for k,v in self.metrics.items():
            res[k + '_tr'] = v(self.dat_tr[1], pred_tr)
            res[k + '_v'] = v(self.dat_v[1], pred_v)
        return res
    
    def score_test(self, model, Xts, yts):
        pred = model.predict(Xts)
        res = {}
        for k,v in self.metrics.items():
            res[k + '_ts'] = v(yts, pred)
        return res

In [95]:
class HyperparamSearch():
    def __init__(self, Xtr, ytr, Xv, yv, score_func):
        self.dat_tr = (Xtr, ytr)
        self.dat_v = (Xv, yv)
        self.f_score = score_func
        
    def tune(self, model_class, args=[], param_var=[], fit_params={}): 
        scores = []
        for p in param_var:
            model = model_class(*args, **p)
            model.fit(*self.dat_tr, **fit_params)
            scores.append(self.f_score(model))
        amax = np.argmax(scores)
        return scores[amax], param_var[amax]

In [96]:
ev_full = Evaluator(Xtr, ytr, Xv, yv, metrics={'acc': accuracy_score, 'conf_m': confusion_matrix})
ev_comp = Evaluator(Xtr, ytr, Xv, yv, metrics={'acc': accuracy_score})

In [97]:
hypp_s = HyperparamSearch(Xtr, ytr, Xv, yv, lambda x: ev_comp.score(x)['acc_v'])

In [133]:
dill.dump_session('./jupyter_session.db')

### implementing softmax regression

In [81]:
from linear_model import SoftmaxRegression, Batcher

In [83]:
batcher = Batcher(256, one_hot=True)
args = (Xtr.shape[1], 10)
param_var = [{'lr': 0.01, 'fit_intercept': True},
             {'lr': 0.05, 'fit_intercept': True},
             {'lr': 0.1, 'fit_intercept': True}]

In [85]:
hypp_s.tune(SoftmaxRegression, args, param_var, {'batch_generator': batcher, 'n_epochs': 100})

HBox(children=(IntProgress(value=0), HTML(value='')))

HBox(children=(IntProgress(value=0), HTML(value='')))

HBox(children=(IntProgress(value=0), HTML(value='')))

(0.9197, {'fit_intercept': True, 'lr': 0.1})

In [86]:
sr = SoftmaxRegression(*args, lr=0.1, fit_intercept=True)
sr.fit(Xtr, ytr, batch_generator=batcher, n_epochs=100)

HBox(children=(IntProgress(value=0), HTML(value='')))

In [98]:
ev_full.score_test(sr, Xts, yts)

{'acc_ts': 0.9168,
 'conf_m_ts': array([[ 959,    0,    2,    3,    0,    6,    8,    1,    1,    0],
        [   0, 1107,    4,    1,    0,    1,    4,    2,   16,    0],
        [   6,   12,  907,   23,   11,    4,   14,   12,   37,    6],
        [   4,    0,   23,  920,    1,   23,    4,    9,   17,    9],
        [   3,    3,    6,    1,  905,    0,    9,    4,   13,   38],
        [   9,    3,    2,   48,    8,  764,   18,    3,   32,    5],
        [  13,    3,    7,    2,   10,   15,  899,    4,    5,    0],
        [   3,    8,   20,   11,    9,    1,    0,  944,    2,   30],
        [   6,   10,   11,   27,    8,   22,   15,   12,  855,    8],
        [   7,    6,    2,    9,   37,    7,    0,   23,   10,  908]])}

In [99]:
ev_full.score_test(sr, Xusps, yusps)

{'acc_ts': 0.36036801840092003,
 'conf_m_ts': array([[ 459,    1,  124,  461,    7,  890,   14,   12,   27,    5],
        [  15, 1372,   67,  151,   18,  179,   31,   51,  112,    4],
        [  43,    6,  841,  548,   31,  361,  118,   19,   29,    3],
        [  19,    5,   94, 1508,   12,  269,    7,   42,   29,   15],
        [  36,   51,  104,   76, 1061,  117,  128,  114,  143,  170],
        [  96,   13,   86,  509,   45,  985,  128,   57,   67,   14],
        [  59,    5,  291,  337,   53,  890,  290,    8,   66,    1],
        [  65,  111,  375,  923,   16,  156,   45,  175,  105,   29],
        [ 119,   35,  131,  637,   27,  594,   89,   14,  347,    7],
        [  48,  141,  122,  577,  181,  119,   39,  461,  143,  169]])}

### SVM

In [14]:
from sklearn.svm import SVC

In [18]:
svm = SVC(kernel='linear', max_iter=500)

In [19]:
svm.fit(Xtr, ytr)

/home/wolterlw/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=500, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

### RandomForest

In [21]:
from sklearn.ensemble import RandomForestClassifier

In [129]:
param_var = [
    {'n_estimators': 50, 'max_depth': 8, 'n_jobs': -1},
    {'n_estimators': 50, 'max_depth': 15, 'n_jobs': -1},
    {'n_estimators': 100, 'max_depth': 8, 'n_jobs': -1},
    {'n_estimators': 100, 'max_depth': 8, 'min_samples_split': 10, 'n_jobs': -1},
    {'n_estimators': 100, 'max_depth': 10, 'min_samples_split': 15, 'n_jobs': -1},
    {'n_estimators': 100, 'max_depth': 15, 'min_samples_split': 15, 'n_jobs': -1},
            ]

In [130]:
hypp_s.tune(RandomForestClassifier, (), param_var, {})

(0.9669, {'max_depth': 15, 'n_estimators': 50, 'n_jobs': -1})

In [134]:
rf = RandomForestClassifier(max_depth=15, n_estimators=50, n_jobs=-1)
rf.fit(Xtr, ytr)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=15, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [135]:
ev_full.score_test(rf, Xts, yts)

{'acc_ts': 0.9623,
 'conf_m_ts': array([[ 972,    1,    0,    0,    0,    3,    1,    1,    2,    0],
        [   0, 1125,    2,    2,    1,    2,    2,    0,    1,    0],
        [   7,    1,  988,   10,    4,    1,    4,   10,    6,    1],
        [   1,    0,   11,  963,    0,   14,    0,   10,    7,    4],
        [   2,    1,    1,    1,  943,    0,    4,    1,    4,   25],
        [   4,    2,    1,   19,    2,  842,    7,    3,    8,    4],
        [   7,    3,    1,    0,    4,    4,  934,    0,    5,    0],
        [   2,    3,   20,    1,    4,    0,    0,  976,    3,   19],
        [   5,    2,    3,   13,    7,    4,    3,    4,  922,   11],
        [   8,    7,    2,    7,   10,    6,    0,    4,    7,  958]])}

In [136]:
ev_full.score_test(rf, Xusps, yusps)

{'acc_ts': 0.5130756537826892,
 'conf_m_ts': array([[1594,   23,   39,   57,    9,  155,   37,   73,    8,    5],
        [   2, 1830,    8,   12,    5,   27,   66,   48,    2,    0],
        [  86,  144, 1130,  118,  103,  193,   61,  157,    7,    0],
        [  24,   54,   53, 1293,   90,  241,    3,  236,    2,    4],
        [  39,  220,   54,   17, 1351,   64,   38,  175,   21,   21],
        [  51,   51,   46,  174,   87, 1310,   49,  222,    8,    2],
        [ 206,   98,  144,   96,  117,  549,  526,  241,   22,    1],
        [  42,  434,  463,  190,   36,   36,   38,  749,    8,    4],
        [  43,  168,  153,  228,  144,  646,  119,   63,  433,    3],
        [  17,  209,   81,  122,  723,   64,   28,  681,   30,   45]])}

### DNN

In [140]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam
from keras.utils import to_categorical

In [138]:
inp = Input(shape=(28*28,))

x = Dense(350, activation='relu', kernel_regularizer='l2')(inp)
x = Dense(170, activation='relu')(x)
out = Dense(10, activation='softmax')(x)

model = Model(inputs=[inp], outputs=[out])

In [139]:
model.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [143]:
class DNN():
    def __init__(self, hidden_sizes=[], activation='relu', regularizers=[], lr=0.01):
        inp = Input(shape=(28*28,))
        x = inp
        for hs,reg in zip(hidden_sizes, regularizers):
            x = Dense(hs, activation=activation, kernel_regularizer=reg)(x)
        out = Dense(10, activation='softmax')(x)
        self.model = Model(inputs=[inp], outputs=[out])
        self.model.compile(Adam(lr), loss='categorical_crossentropy', metrics=['accuracy'])
    
    def fit(self, X, y, **kwargs):
        yoh = to_categorical(y)
        self.model.fit(X, yoh, **kwargs)
    
    def predict(self, X):
        return np.argmax(
            self.model.predict(X),
            axis=1
        )

In [144]:
dnn = DNN([350, 170], regularizers=['l2',None])

In [147]:
param_var = [
    {'hidden_sizes': [100, 50], 'activation': 'relu', 'regularizers': [None, None]},
    {'hidden_sizes': [250, 100], 'activation': 'relu', 'regularizers': [None, None]},
    {'hidden_sizes': [250, 100], 'activation': 'relu', 'regularizers': ['l2', None]},
    {'hidden_sizes': [350, 150], 'activation': 'relu', 'regularizers': ['l2', None]},
    {'hidden_sizes': [350, 150], 'activation': 'relu', 'regularizers': ['l2', 'l2']},
            ]

In [148]:
hypp_s.tune(DNN, param_var=param_var, fit_params={'batch_size': 512, 'epochs': 30, 'verbose': False})

(0.9754,
 {'activation': 'relu',
  'hidden_sizes': [100, 50],
  'regularizers': [None, None]})

### CNN

#### LeNet-5

In [162]:
from keras.layers import Conv2D, BatchNormalization, MaxPool2D, ReLU, Concatenate, Dropout, Flatten

In [169]:
class LeNet5():
    def __init__(self):
        inp = Input(shape=(28,28,1))
        x = Conv2D(6, kernel_size=5, strides = 1, activation = 'relu')(inp)
        x = MaxPool2D(2, strides = 2)(x)
        x = Conv2D(16, kernel_size=5, strides = 1, activation = 'relu')(x)
        x = MaxPool2D(2, strides=2)(x)
        x = Flatten()(x)
        x = Dense(120, activation='relu')(x)
        x = Dense(84, activation='relu')(x)
        out = Dense(10, activation='softmax')(x)

        self.model = Model(inputs=[inp], outputs=[out])
        self.model.compile('Adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    def fit(self, X, y, **kwargs):
        yoh = to_categorical(y)
        Xim = X.reshape(-1,28,28,1)
        self.model.fit(Xim, yoh, **kwargs)
    
    def predict(self, X):
        return np.argmax(
            self.model.predict(X.reshape(-1,28,28,1)),
            axis = 1
        )

In [170]:
lenet = LeNet5()

In [172]:
lenet.fit(Xtr, ytr, batch_size=512, epochs=30, validation_data=(Xv.reshape(-1,28,28,1), to_categorical(yv)))

Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 27s 548us/step - loss: 0.2500 - acc: 0.9269 - val_loss: 0.1810 - val_acc: 0.9494
Epoch 2/30
50000/50000 [==============================] - 27s 548us/step - loss: 0.1630 - acc: 0.9509 - val_loss: 0.1340 - val_acc: 0.9611
Epoch 3/30
50000/50000 [==============================] - 27s 545us/step - loss: 0.1263 - acc: 0.9624 - val_loss: 0.1116 - val_acc: 0.9673
Epoch 4/30
50000/50000 [==============================] - 27s 545us/step - loss: 0.1051 - acc: 0.9684 - val_loss: 0.1049 - val_acc: 0.9694
Epoch 5/30
50000/50000 [==============================] - 28s 553us/step - loss: 0.0896 - acc: 0.9729 - val_loss: 0.0846 - val_acc: 0.9737
Epoch 6/30
50000/50000 [==============================] - 27s 547us/step - loss: 0.0809 - acc: 0.9753 - val_loss: 0.0819 - val_acc: 0.9754
Epoch 7/30
50000/50000 [==============================] - 27s 548us/step - loss: 0.0704 - acc: 0.9784 - val_loss: 0.

In [173]:
ev_full.score_test(lenet, Xts, yts)

{'acc_ts': 0.9891,
 'conf_m_ts': array([[ 971,    0,    0,    0,    0,    0,    3,    2,    2,    2],
        [   0, 1130,    2,    0,    0,    0,    2,    1,    0,    0],
        [   1,    1, 1025,    1,    0,    0,    1,    1,    2,    0],
        [   0,    0,    1, 1005,    0,    4,    0,    0,    0,    0],
        [   0,    0,    0,    0,  970,    0,    1,    2,    0,    9],
        [   1,    0,    0,    7,    0,  880,    2,    1,    0,    1],
        [   4,    2,    0,    0,    1,    3,  947,    0,    1,    0],
        [   1,    2,    8,    2,    0,    0,    0, 1014,    0,    1],
        [   3,    0,    5,    3,    0,    1,    0,    1,  961,    0],
        [   1,    1,    1,    6,    3,    3,    0,    2,    4,  988]])}

In [174]:
ev_full.score_test(lenet, Xusps, yusps)

{'acc_ts': 0.8288414420721036,
 'conf_m_ts': array([[1858,    2,   27,    6,    0,   16,   36,    3,   33,   19],
        [  13, 1954,    5,    0,    0,    0,    8,    7,   12,    1],
        [  25,   17, 1698,  102,    2,   84,   23,    7,   39,    2],
        [   3,    4,   18, 1909,    3,   54,    0,    3,    4,    2],
        [  11,   13,   21,    6, 1724,    4,   46,  104,   47,   24],
        [  22,    5,   13,   92,    2, 1680,   17,   72,   72,   25],
        [  67,    5,   56,    3,    0,   22, 1808,    0,   39,    0],
        [  19,  143,  195,   84,   12,    5,    0, 1512,   21,    9],
        [  11,    8,   40,   64,   13,   87,   17,   43, 1712,    5],
        [   3,   34,    9,   35,  419,    4,   10,  662,  103,  721]])}

#### DenseNet-ish thing

In [154]:
def dense_block(filters=32, kernel_size=3):
    def f(x):
        x0 = BatchNormalization()(x)
        x0 = ReLU()(x0)
        x0 = Conv2D(filters, kernel_size=3, padding='same')(x0)
        x0 = Dropout(0.2)(x0)
        x0 = BatchNormalization()(x0)
        x0 = ReLU()(x0)
        x = Concatenate()([x, x0])
        return x
    return f
    
def bottleneck(filters=32):
    def f(x):
        x = BatchNormalization()(x)
        x = Conv2D(filters, kernel_size=1, padding='same')(x)
        x = MaxPool2D()(x)
        return x
    return f

In [186]:
class DenseNet():
    def __init__(self, filters=[], dense_layers=[]):
        inp = Input(shape=(28, 28, 1))
        x = inp
        for flt in filters:
            x = dense_block(flt, 3)(x)
            x = bottleneck(flt//2)(x)
        
        x = Flatten()(x)
        
        for dl in dense_layers:
            x = Dense(dl, activation='relu')(x)

        out = Dense(10, activation='softmax')(x)
        
        self.model = Model(inputs=[inp], outputs=[out])
        self.model.compile('Adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    def fit(self, X, y, **kwargs):
        yoh = to_categorical(y)
        Xim = X.reshape(-1,28,28,1)
        self.model.fit(Xim, yoh, **kwargs)
    
    def predict(self, X):
        return np.argmax(
            self.model.predict(X.reshape(-1,28,28,1)),
            axis = 1
        )

inp = Input(shape=(28, 28,1))

x = dense_block(64, 3)(inp)
x = bottleneck(32)(x)
x = dense_block(32, 3)(x)
x = bottleneck(16)(x)
x = dense_block(16, 3)
x  = bottl

In [190]:
densenet = DenseNet(filters = [32, 16], dense_layers=[32])

In [191]:
densenet.fit(Xtr, ytr, 
             batch_size=512, epochs=30,
             validation_data=(Xv.reshape(-1,28,28,1), to_categorical(yv)))

Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 372s 7ms/step - loss: 1.0190 - acc: 0.6732 - val_loss: 0.3033 - val_acc: 0.9092
Epoch 2/30
50000/50000 [==============================] - 370s 7ms/step - loss: 0.2569 - acc: 0.9203 - val_loss: 0.1759 - val_acc: 0.9488
Epoch 3/30
50000/50000 [==============================] - 368s 7ms/step - loss: 0.1646 - acc: 0.9489 - val_loss: 0.1401 - val_acc: 0.9591
Epoch 4/30
50000/50000 [==============================] - 368s 7ms/step - loss: 0.1229 - acc: 0.9617 - val_loss: 0.1027 - val_acc: 0.9698
Epoch 5/30
50000/50000 [==============================] - 368s 7ms/step - loss: 0.1015 - acc: 0.9684 - val_loss: 0.0843 - val_acc: 0.9754
Epoch 6/30
50000/50000 [==============================] - 367s 7ms/step - loss: 0.0879 - acc: 0.9722 - val_loss: 0.0798 - val_acc: 0.9770
Epoch 7/30
50000/50000 [==============================] - 368s 7ms/step - loss: 0.0767 - acc: 0.9758 - val_loss: 0.0848 - 

In [192]:
ev_full.score_test(densenet, Xts, yts)

{'acc_ts': 0.9868,
 'conf_m_ts': array([[ 978,    0,    0,    0,    0,    0,    1,    1,    0,    0],
        [   0, 1133,    1,    1,    0,    0,    0,    0,    0,    0],
        [   2,    5, 1022,    0,    0,    0,    0,    3,    0,    0],
        [   0,    1,    5,  994,    0,    8,    0,    0,    1,    1],
        [   0,    1,    0,    0,  971,    0,    4,    2,    0,    4],
        [   1,    0,    1,    3,    0,  885,    1,    0,    0,    1],
        [   8,    4,    1,    0,    2,    3,  940,    0,    0,    0],
        [   0,    5,    6,    0,    0,    1,    0, 1015,    0,    1],
        [   8,    3,    4,    3,    1,    2,    2,    3,  941,    7],
        [   2,    3,    2,    1,    4,    4,    0,    4,    0,  989]])}

In [193]:
ev_full.score_test(densenet, Xusps, yusps)

{'acc_ts': 0.8177908895444772,
 'conf_m_ts': array([[1747,   13,  121,    7,    5,   22,   28,    5,   36,   16],
        [   1, 1979,    4,    0,    1,    2,    3,   10,    0,    0],
        [   3,   18, 1807,   55,    1,   64,    1,   10,   38,    2],
        [   1,    1,   39, 1814,    0,  137,    1,    5,    2,    0],
        [   7,   66,   25,    2, 1751,    4,   25,   23,   69,   28],
        [   0,    4,   10,  112,    1, 1844,    4,   19,    1,    5],
        [  42,   56,  172,   10,    1,  189, 1427,    7,   94,    2],
        [   0,  267,  603,   44,    8,   10,    1, 1040,   12,   15],
        [   1,    7,   38,   42,    6,  205,    9,    5, 1674,   13],
        [   6,   64,  150,   67,  190,   26,    4,  134,   87, 1272]])}

### Ensambling

In [194]:
models = [densenet, sr, lenet, rf, dnn]

In [221]:
class MajorityVoting():
    def __init__(self, classifiers=[]):
        self.clfs = classifiers
    
    def predict(self, X):
        preds = np.c_[[clf.predict(X) for clf in self.clfs]]
        return mode(preds.T, axis=1)[0][:,0]

In [200]:
from scipy.stats import mode

In [222]:
mv = MajorityVoting(models)

In [228]:
pred_usps = mv.predict(Xusps)

In [229]:
(pred_usps == yusps).mean()

0.6741837091854592

In [233]:
models = [densenet, lenet]

In [234]:
mv2 = MajorityVoting(models)
pred_usps = mv.predict(Xusps)

In [237]:
(lenet.predict(Xusps) == yusps).mean()

0.8288414420721036

In [235]:
(pred_usps == yusps).mean()

0.6741837091854592